<a href="https://colab.research.google.com/github/tsakailab/cisexpkit/blob/master/Experiment/colab/pointcloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 逆透視変換による3次元復元

### 視差画像の例を取得して表示します．
[Middlebury Stereo Datasets](http://vision.middlebury.edu/stereo/data/) の [2005 Datasets](http://vision.middlebury.edu/stereo/data/scenes2005/) を使います．

In [0]:
MB05_names = ["Art", "Books", "Dolls", "Laundry", "Moebius", "Reindeer"]
MB05_name = MB05_names[  0  ]  # choose 0 to 5
MB05_scales = [("FullSize", 1.0), ("HalfSize", 2.0), ("ThirdSize", 3.0)]
MB05_scale = MB05_scales[2]  # choose 0 to 2
zipURL = "http://vision.middlebury.edu/stereo/data/scenes2005/" + MB05_scale[0] + "/zip-2views/" + MB05_name +"-2views.zip"

print("Downloading the dataset " + MB05_name + " (" + MB05_scale[0] + ") ..")
!wget $zipURL --no-check-certificate --show-progress -q -O "/tmp/tmp.zip"

In [0]:
import zipfile
import os

with zipfile.ZipFile("/tmp/tmp.zip", 'r') as f:
    f.extractall("/tmp")

import numpy as np
from PIL import Image

root_dir = "/tmp/" + MB05_name
left = np.asarray(Image.open(root_dir + '/view1.png'))
right = np.asarray(Image.open(root_dir + '/view5.png'))
disp1 = np.asarray(Image.open(root_dir + '/disp1.png')) / MB05_scale[1]
height, width, colors = left.shape
with open(root_dir + '/dmin.txt', 'r') as f:
    doffs = int(f.read()) / MB05_scale[1]

%matplotlib inline
import matplotlib.pyplot as plt
#"""
plt.figure(figsize=(15,4))
plt.subplot(1,3,1)
plt.imshow(left)
plt.subplot(1,3,2)
plt.imshow(right)
plt.subplot(1,3,3)
#"""
plt.imshow(disp1, cmap="gray")

## 逆透視変換で点群を作りましょう．
まず，各画素にuとvの座標を設定します．座標系は[予習事項](https://github.com/tsakailab/cisexpkit/blob/master/Experiment/colab/preparation.pdf)の図1です．

In [0]:
# 画像の中心を主点と仮定します．
cx, cy = width*0.5, height*0.5
j_to_u = lambda j: -(j - cx) * MB05_scale[1]
i_to_v = lambda i: -(i - cy) * MB05_scale[1]
u, v = np.meshgrid(j_to_u(np.arange(width)), i_to_v(np.arange(height)))
print(u, u.shape)
print(v, v.shape)

Z, u, vが与えられたとき，X[mm]とY[mm]を計算する関数を作りましょう．[ヒント：予習事項の問2](https://github.com/tsakailab/cisexpkit/blob/master/Experiment/colab/preparation.pdf)

Q11: 関数 Zuv_to_XYを完成させよ．単位mmの深度Zと，画素数の単位をもつ座標u, v, 焦点距離fが与えられているものとする．

In [0]:
focal_length = 3740 # 焦点距離 f [pixels]

def Zuv_to_XY(Z, u, v, f=focal_length):
    ### X = ________   # Z, u, v, f から必要なものを使って計算する
    ### Y = ________   # Z, u, v, f から必要なものを使って計算する
    return X, Y

次に，視差 d から3次元座標(X,Y,Z)の深度Zを計算する式を書きましょう．
ただし，焦点距離をfocal_length，基線の長さを baseline とします．

（実験(a)1日目のQ10）

Q12: 基線の長さ baseline [mm], 焦点距離 focal_length [pixels], 視差 d [pixels]，左右画像の主点の差 doffs [mm] が与えられている．
深度 Z [mm] を計算する式を作成せよ．
> ヒント: [Middlebury 2014 stereo datasets](http://vision.middlebury.edu/stereo/data/scenes2014/#description)

In [0]:
baseline = 120 # 基線長l [mm]
d = disp1 # 視差 d [pixels]

### Z = ________  # baseline, focal_length, d, doffs を使って計算する（for文不要）．

# Z, u, v から X, Y を計算します．
X, Y = Zuv_to_XY(Z, u, v)

## ポイントクラウドを可視化して観察しましょう．

Q13: 使用した画像のサイズと全画素数はいくらか．全画素のうち，何画素が逆透視変換可能か．逆透視変換できない点が生じる原因を述べよ．

Q14: [plotly](https://plotly.com/)とは何か．特に，[plotly Graphing Libraries](https://plotly.com/graphing-libraries/)はどのような特長があるか．

Q15: [plotly.graph_objects.Scatter3d](https://plotly.com/python-api-reference/generated/plotly.graph_objects.Scatter3d.html#plotly.graph_objects.Scatter3d)は何ができる関数で，このサンプルコードではどのように使われているか．

> マウスで視点や拡大・縮小をコントロールできます．描画領域右上にある機能も活用しましょう．

> 設定を変えて再描画後に表示されなくなった場合は，ブラウザでページを開きなおしてください．


In [0]:
nd = np.count_nonzero(d)
n = 20000
p = np.random.choice(nd, min(n,nd), replace=False)
print("%d out of %d points are displayed." % (n, nd))

import plotly.graph_objs  as go
trace = go.Scatter3d(x=X[d>0][p], y=Y[d>0][p], z=Z[d>0][p], mode='markers',
                     marker=dict(size=2, 
                                color=['rgb({},{},{})'.format(r,g,b) for r,g,b in zip(left[:,:,0][d>0][p], left[:,:,1][d>0][p], left[:,:,2][d>0][p])],
                                opacity=0.8))

layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))
fig = go.Figure(data=[trace], layout=layout)
camera = dict(up=dict(x=0, y=0, z=1), center=dict(x=0, y=-0.4, z=0), eye=dict(x=0, y=0.8, z=-2))
fig.update_layout(scene_camera=camera)
fig.show()